In [ ]:
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import matplotlib.ticker as tick
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
from pathlib import Path
from pydash import py_
from green_city.utils import datetime2index, index2datetime

from green_city.mlflow_config import get_mlflow_config
mlflow.set_tracking_uri(get_mlflow_config()["TRACKING_URI"])
client = MlflowClient()
experiment_id = client.get_experiment_by_name("green_city_experiments").experiment_id

## DB CONNECTION ##
from sqlalchemy import create_engine
from decouple import Config, RepositoryEnv

config = Config(RepositoryEnv("../.db_credentials"))

db_connection_credentials = {
    "database": config('POSTGRES_DB'),
    "user": config('POSTGRES_USER'),
    "password": config('POSTGRES_PASSWORD'),
    "host": config('POSTGRES_HOST'),
    "port": config('POSTGRES_PORT'),
}
DB_STRING = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_connection_credentials)
db = create_engine(DB_STRING)

In [ ]:
#global configuration
BUILDING_NR = "all"
USE_ROOT = True #RMSE insead of MSE

indices = [32135, 33311, 26478, 33357, 30387, 30794, 31800, 28783]
INDEX_LINEPLOT = indices[7]

##########################################################
#df.loc[df["run_name"] == "Prophet Simple Model", "model"] = "prophet"
#df.loc[df["run_name"] == "Prophet Simple Model Optimized Parameters", "model"] = "prophet"
#    if df_row["run_name"] == "Prophet Simple Model":
#        return "Prophet"
#df[df["run_name"] == "Prophet Simple Model"].index
##########################################################
#    plot_df = plot_df[plot_df["run_name"] != "sarimax_(6,1,1)(0,0,2,24)"]
#    plot_df = plot_df[plot_df["disp_label"].map(lambda x: x in ("Linear regression", "Sarimax", "Prophet", "TBATS", "ARIMA", "Baseline"))]
##########################################################


selectors = {
    'baseline': {
        'select__model': 'baseline',
        'disp_color': 'orange',
        'disp_label': 'Baseline',
    },
    'linear_regression': {
        'select__run_name': 'linear_regression', #wouldn't need both selectors actually
        'select__model': 'linear_regression',
        'disp_color': '#7BC8F6',
        'disp_label': 'Linear regression',
    },
    'tbats': {
        'select__run_name': 'TBATS_1000',
        'select__model': 'TBATS',
        'disp_color': 'blue',
        'disp_label': 'TBATS',
    },
    #'arima': {
    #    'select__model': 'ARIMA',
    #    'disp_color': 'purple',
    #    'disp_label': 'ARIMA',
    #},
    'sarimax': {
        'select__model': 'sarimax',
        'exclude__run_name': 'sarimax_(6,1,1)(0,0,2,24)',
        'disp_color': '#FF7F50',
        'disp_label': 'SARIMAX',
    },
    'prophet_1': {
        'select__run_name': 'Prophet Holidays-Wkl-Regressors-Optimized-Parameters - Tunning with MSE',
        'disp_color': 'purple',
        'disp_label': 'Prophet',
    },
    #'prophet_2': {
    #    'select__run_name': 'Prophet Holidays-Regressors-Optimized-Parameters - Tunning with MSE',
    #    'disp_color': 'purple',
    #    'disp_label': 'Prophet 2',
    #},
    #'prophet_3': {
    #    'select__run_name': 'Prophet Holidays-Optimized-Parameters - Tunning with MSE',
    #    'disp_color': 'purple',
    #    'disp_label': 'Prophet 3',
    #},
    #'prophet_4': {
    #    'select__run_name': 'Prophet Simple Model Optimized Parameters - Tunning with MSE',
    #    'disp_color': 'purple',
    #    'disp_label': 'Prophet 4',
    #},
    #'prophet_5': {
    #    'select__run_name': 'Prophet Simple Model',
    #    'disp_color': 'purple',
    #    'disp_label': 'Prophet 5',
    #},
    'random_forest': {
        'select__run_name': 'random_forest',
        'disp_color': 'green',
        'disp_label': 'Random forest',
    },
    'poly_regression': {
        'select__run_name': 'poly_regression',
        'disp_color': '#FFD700',
        'disp_label': 'Polynomial regression',
    },
    'xgb_optimized': {
        'select__run_name': 'XGB_optimized',
        'disp_color': '#06C2AC',
        'disp_label': 'XGBoost',
    },
    #'xgb_simple': {
    #    'select__run_name': 'XGB_simple',
    #    'disp_color': 'cyan',
    #    'disp_label': 'XGB simple',
    #},
}

def select(run_info):
    for name, di in selectors.items():
        is_match = True
        for select_key in filter(lambda x: 'select__' in x, di.keys()):
            key = select_key.split('__')[1]
            if run_info.get(key) != di[select_key]:
                is_match = False
                break
        for exclude_key in filter(lambda x: 'exclude__' in x, di.keys()):
            key = exclude_key.split('__')[1]
            if run_info.get(key) == di[exclude_key]:
                is_match = False
                break
        if is_match:
            return (name, dict([(k, v) for k, v in di.items() if k in ['disp_color', 'disp_label'] ]))
    return (None, None)


In [ ]:
run_ids = [r.run_id for r in client.list_run_infos(experiment_id, run_view_type=ViewType.ACTIVE_ONLY)]

all_runs = {run_id: client.get_run(run_id).to_dictionary() for run_id in run_ids}

In [ ]:
runs_dict = {}

for id, details in all_runs.items():
    start_time = py_.get(details, "info.start_time")
    end_time = py_.get(details, "info.end_time")
    #delta = round(end_time/1000 - start_time/1000, 2)

    #filter by buildings
    params = py_.get(details, "data.params")

    if len(params) == 0: #probably currently active run
        continue
    
    runs_dict_id = {
        #"time_s": delta,
        **py_.get(details, "data.metrics"),
        **py_.get(details, "data.params"),
        "run_name": details["data"]["tags"]["mlflow.runName"],
    }

    if (runs_dict_id.get("building nr") and not runs_dict_id.get("building_nr")):
        runs_dict_id["building_nr"] = runs_dict_id.pop("building nr")
    
    assert 'building nr' not in runs_dict_id.keys()
    assert 'building_nr' in runs_dict_id.keys()

    #selection 1: building nr
    if BUILDING_NR is not None:
        if BUILDING_NR.lower() != runs_dict_id['building_nr'].lower():
            continue

    #selection 2: models to include
    selection_name, additional_properties = select(runs_dict_id)
    if selection_name is not None:
        runs_dict_id.update(additional_properties)
        runs_dict_id["name"] = selection_name
    else:
        continue

    runs_dict[id] = runs_dict_id

In [ ]:
df = pd.DataFrame.from_dict(runs_dict, orient='index')
df = df.astype({"datetime": "datetime64[ns]"})
df = df.assign(index = df['index'].fillna(df.datetime.map(datetime2index))).dropna(subset=["index"])

#if index is not set, assign it from datetime
df.datetime.map(datetime2index).unique()
df = df.astype({"index": "int", "train_steps": "float"})
df.index = df.index.rename("run_id")

In [ ]:
print(len(df))
df

In [ ]:
#plt.rcParams['figure.figsize'] = [25, 8]
#df[df.feature=="net_load_kWh"].loc[:, ["mae", "mse", "r2_score"]].plot.barh()
#df[df.feature=="net_load_kWh"].loc[:, ["time_s"]].plot.barh()
df.to_csv("../data/results/evaluation_matrix.csv")

In [ ]:
plt.rcParams['figure.figsize'] = [12, 4]

#TODO: take keys from selection dictionary or from actual used names.
#all_performances = {
#    "baseline": [],
#    "linear_regression": [],
#    "TBATS": [],
#    "sarimax": [],
#    "ARIMA": [],
#    "prophet": [],
#}
all_performances = {k: [] for k in selectors.keys()}

all_performances_dicts = []


for index in indices:
    fig, ax = plt.subplots()
    fig.set_figheight(2)
    fig.set_figwidth(5)
    fig.set_dpi(240)

    plot_df = (df[df["index"] == index]
                .sort_values("mse", ascending=False)
                #.assign(col = lambda x:x["model"].map(lambda x: model2color.get(x, "black")))
                #.assign(label = lambda x: f"{x['model']}{steps2str(x['train_steps'])}")
                .drop_duplicates(subset=["run_name", "model"])
    )
    plot_df["rmse"] = np.sqrt(plot_df["mse"])

    
    for idx, benchmark in plot_df.iterrows():
        all_performances[benchmark["name"]].append(benchmark["mse"])
        all_performances_dicts.append({
            'model': [benchmark['name']],
            'time_index': [index],
            'run_index': [idx],
            'mse': [benchmark['mse']],
            'mae': [benchmark['mae']],
            'rmse': [benchmark['rmse']],
        })
    
    if USE_ROOT:
        score_col = "rmse"
    else:
        score_col = "mse"
    plot_df.plot.barh(x="disp_label", y=score_col, color=plot_df["disp_color"], legend=None, ax=ax)
    #ax.barh(plot_df["mse"], width=20, color=plot_df["col"], legend=None, height=0.9)
    axis_label = "Root Mean Squared Error" if USE_ROOT else "Mean squared error"
    ax.set_xlabel(axis_label)
    ax.set_ylabel(None)
    #fig.suptitle(f"{str(index2datetime(index+1))} to {str(index2datetime(index+24))}")
    fig.suptitle("")
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

all_performances_df = pd.DataFrame({
    'model': [],
    'time_index': [],
    'mse': [],
    'mae': [],
    'rmse': [],
})
all_performances_df = pd.concat([pd.DataFrame(di) for di in all_performances_dicts], ignore_index=True)
all_performances_df

In [ ]:
all_performances_df["disp_color"] = all_performances_df.model.map(lambda x: selectors[x]["disp_color"])
all_performances_df["disp_label"] = all_performances_df.model.map(lambda x: selectors[x]["disp_label"])
all_performances_df

In [ ]:
from matplotlib.pyplot import xlabel

agg_performances = all_performances_df.groupby("model").mean().sort_values("rmse", ascending=False)
agg_performances["disp_color"] = agg_performances.index.map(lambda x: selectors[x]["disp_color"])
agg_performances["disp_label"] =  agg_performances.index.map(lambda x: selectors[x]["disp_label"])
baseline_score = agg_performances[agg_performances.index == "baseline"]["rmse"].values[0]
baseline_score_mse = agg_performances[agg_performances.index == "baseline"]["mse"].values[0]
baseline_score_mae = agg_performances[agg_performances.index == "baseline"]["mae"].values[0]
agg_performances["relative_rmse"] = agg_performances.rmse.map(lambda x: x/baseline_score)
agg_performances["relative_mse"] = agg_performances.mse.map(lambda x: x/baseline_score_mse)
agg_performances["relative_mae"] = agg_performances.mae.map(lambda x: x/baseline_score_mae)
agg_performances
fig, ax = plt.subplots(figsize=(10,4))
agg_performances.plot.barh(
    x="disp_label",
    #y="relative_mse",
    y="relative_mae",
    #label="relative_rmse",
    color=agg_performances.disp_color,
    title="Mean absolute error (lower is better)",
    xlabel="",
    legend=None,
    ax=ax);
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

for i, (model_index, model_info) in enumerate(agg_performances.iterrows()):
    if model_index == "baseline":
        continue
    x = model_info.relative_mae
    y = i
    #ax.text(x+0.01, y-0.1, f"{100*model_info.relative_mae:.0f}%", color=model_info.disp_color, size=14)
    ax.text(x+0.01, y-0.18, f"{100*model_info.relative_mae:.0f}%", color="black", size=12)

fig.set_dpi(250)

In [ ]:
avg_performance = {}
for k, v in all_performances.items():
    #if k == "ARIMA":
    #    continue
    #print(k, v)
    #avg_performance[k] = [0]
    avg_performance[k] = np.mean(v) #TODO: check if I am overwriting sth here
mmses = pd.concat([pd.DataFrame({"model": [k], "mmse": [v]}) for k, v in avg_performance.items()], axis=0)
mmses = mmses.sort_values("mmse", ascending=False)

#mmses["color"] = mmses.model.map(lambda x: model2color.get(x, "black"))
mmses["color"] = mmses.model.map(lambda x: selectors[x]["disp_color"])

fig, ax = plt.subplots()
fig.set_figheight(4)
fig.set_figwidth(9)
fig.set_dpi(240)

mmses["disp_label"] = mmses.model.map(lambda x: selectors[x]["disp_label"])

mmses.plot.barh(x="disp_label", y="mmse", color=mmses["color"], ax=ax)
ax.set_xlabel("Average performance (Mean Squared Error)")
ax.set_ylabel("")
ax.legend().remove()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)

In [ ]:
# Load the actual dataset
def get_df(building_nr, feature):
    if building_nr == "all":
        filename = "Agg_buildings.csv"
    else:
        filename = f"Building_{building_nr}.csv"
    df = (
    pd.read_csv(Path("../data/preprocessed") / filename)
        .astype({'datetime': 'datetime64'})
        [[feature, "datetime"]]
        .rename(columns={feature: "actual"})
    )
    df.index.name = "id"
    return df

actual_df = get_df(BUILDING_NR, "net_load_kW")

In [ ]:
plt.rcParams['figure.figsize'] = [25, 6]

#index_to_plot = INDEX_LINEPLOT
index_to_plot = indices[2] #2 or 3
#index_to_plot = indices[1]
plot_performances = all_performances_df[all_performances_df["time_index"] == index_to_plot]
plot_df = df[df["index"] == index_to_plot].drop_duplicates(subset=["run_name"])
#display(plot_df)
#display(plot_performances)
#print(plot_performances.shape)
#display(all_performances_df)
#print(all_performances_df.shape)
from pandas import concat

df_from_csv_1 = pd.read_csv("../data/results/linear_regression.csv")
df_from_csv_2 = pd.read_csv("../data/results/sarimax_(2,1,1)(1,0,1,24).csv")
df_from_csv = pd.concat([df_from_csv_1, df_from_csv_2], axis=0, ignore_index=True)[["runid", "id", "prediction"]].rename(columns={"runid": "run_id"})
#df_from_csv.head()

y_true = actual_df.loc[index_to_plot+1:index_to_plot+24].set_index("datetime")

fig = plt.figure(figsize=(11,3), dpi=240)
ax = fig.add_subplot(111)

for run_id, run_data in plot_performances.set_index("run_index").iterrows():
    if not run_data["model"] in ["random_forest", "prophet_1"]:
        continue
#for run_id, run_data in plot_df.iterrows():
    plot_kwargs = {}
    query_string = f"SELECT * FROM forecast WHERE run_id='{run_id}'" # LIMIT 10"
    run_predictions = pd.read_sql(query_string, db).drop(columns="run_id")
    if len(run_predictions) == 0:
        print(f"[INFO: for {run_id}, there is not data in the SQL database.]")
        #print(f"[...trying to read from csv file]")
        run_predictions = df_from_csv[df_from_csv.run_id == run_id].copy().drop(columns="run_id")
        if len(run_predictions) == 0:
            print(f"[Warning: didn't find {run_id} csv data]")
            continue
        #TODO: include plot style in selectors dict
        #if run_data.run_name in ["linear_regression", "sarimax_(2,1,1)(1,0,1,24)"]:
        if run_data.model in ["linear_regression", "sarimax_(2,1,1)(1,0,1,24)"]:
            plot_kwargs = {
                'linestyle': 'dashed'
            }

    run_predictions["datetime"] = run_predictions["id"].map(index2datetime)#y_true.index
    #color = selectors[run_data['name']].get("disp_color")
    ax.plot(run_predictions.drop(columns=["id"]).set_index("datetime"), color=run_data["disp_color"], label=run_data["disp_label"], **plot_kwargs)

ax.plot(y_true, color="black", linestyle="dashed", label="Actual data");
#plt.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))
ax.legend(prop={'size': 9})

def y_fmt(x, y):
    return f"{int(x)} kW"

#ax.set_ylim(21, 82)
#ax.set_ylim(0, 62)
#if BUILDING_NR == 'all':
#    ax.set_ylim(-50, 500)
ax.yaxis.set_major_formatter(tick.FuncFormatter(y_fmt))
date_form = DateFormatter("%H:%M")
ax.xaxis.set_major_formatter(date_form)
#fig.suptitle("Net energy usage forecasts (Year 4, Sep 2)")
fig.suptitle("Energy demand forecasts (Year 4, Apr 15 - 16)")
ax.legend(bbox_to_anchor=(1,0), loc="lower left")